# Ngrams & Dependency Analysis

In [ ]:
import json
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import pandas as pd
import pyrootutils
import seaborn as sns

from formal_gym import grammar as fg_grammar

In [ ]:
PROJECT_ROOT = pyrootutils.find_root(
    search_from=os.path.abspath(""), indicator=".project-root"
)

grammars_dir = PROJECT_ROOT / "data" / "grammars"
grammar_stats_filename = "grammar_stats.json"
samples_stats_filename = "filtered_samples_stats.json"

grammar_dirs = [
    f
    for f in grammars_dir.iterdir()
    if (f.is_dir())
    and (f / grammar_stats_filename).exists()
    and (f / samples_stats_filename).exists()
]

grammar_stats = []
for g in grammar_dirs:
    g_stats = json.load(open(g / grammar_stats_filename))
    s_stats = json.load(open(g / samples_stats_filename))
    merged = {**g_stats, **s_stats}
    grammar_stats.append(merged)
grammar_stats_df = pd.DataFrame(grammar_stats)

grammar_stats_df = (
    grammar_stats_df[
        (grammar_stats_df.coverage > 0.98)
        & (grammar_stats_df.n_terminals <= 100)
        & (grammar_stats_df.n_nonterminals <= 100)
        & (grammar_stats_df.n_lexical_productions <= 100)
        & (grammar_stats_df.n_nonlexical_productions <= 100)
    ]
    .drop(["median_positive_parses", "mean_positive_parses"], axis=1)
    .sort_values(by="grammar_name", ascending=True)
    .reset_index(drop=True)
)

# grammar_stats_df["grammar"] = grammar_stats_df["grammar_name"].apply(
#     lambda x: fg_grammar.Grammar.from_file(grammars_dir / x / f"{x}.cfg")
# )

grammar_stats_df